In [1]:
import pandas as pd
import numpy as np
from sklearn import *
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder


In [2]:
columns_name = ['age','workclass','fnlwgt','education','education_num','marital-status',
               'occupation','relationship','race','sex','capital-gain','capital-loss',
               'hours-per-week','native-country']
df = pd.read_csv('train-features.csv',names=columns_name,header=None,na_values=' ?')
df['income'] = pd.read_csv('train-output.csv') #Binary (0 means <=50K, 1 means >50K)
df.columns = df.columns.str.replace('-','_')

In [260]:
df.hours_per_week.unique()

array([40, 13, 16, 45, 50, 80, 30, 35, 60, 20, 52, 44, 15, 25, 38, 43, 55,
       48, 58, 32, 70,  2, 22, 56, 41, 28, 36, 24, 46, 42, 12, 65,  1, 10,
       34, 75, 98, 33, 54,  8,  6, 64, 19, 18, 72,  5,  9, 47, 37, 21, 26,
       14,  4, 59,  7, 99, 53, 39, 62, 57, 78, 90, 66, 11, 49, 84,  3, 17,
       68, 27, 85, 31, 51, 77, 63, 23, 87, 88, 73, 89, 97, 94, 29, 96, 67,
       82, 86, 91, 81, 76, 92, 61, 74, 95])

## Classification

https://medium.com/datadriveninvestor/k-fold-cross-validation-6b8518070833

In [310]:
class Classification:
    def __init__(self):
        
        self.model = None
        self.columns_name = ['age','workclass','fnlwgt','education','education_num',
                                       'marital_status','occupation','relationship','race','sex',
                                       'capital_gain','capital_loss','hours_per_week','native_country']
        self.train_data = pd.read_csv('train-features.csv',names=self.columns_name,header=None,na_values=' ?')
        self.train_label = pd.read_csv('train-output.csv',names=['income'],header=None) #Binary (0 means <=50K, 1 means >50K)
#         self.train_label = self.train_label.values.ravel()
        self.test_data = pd.read_csv('test-features.csv',names=self.columns_name,header=None,na_values=' ?')
        
        self.x_train = None
        self.x_test  = None
        self.y_train = None
        self.y_test  = None
    def preprocess(self,df,bins):
        '''
        Cleans df and performs feature engineering.
        '''
        # replace nan values with most commone values of it's column
        for i in self.columns_name:
            df[i] = df[i].fillna(list(dict(df_data['education'].dropna().value_counts()))[0])

        # category
        df['education'] = df['education'].str.replace('Preschool', 'dropout')
        df['education'] = df['education'].str.replace('10th', 'dropout')
        df['education'] = df['education'].str.replace('11th', 'dropout')
        df['education'] = df['education'].str.replace('12th', 'dropout')
        df['education'] = df['education'].str.replace('1st-4th', 'dropout')
        df['education'] = df['education'].str.replace('5th-6th', 'dropout')
        df['education'] = df['education'].str.replace('7th-8th', 'dropout')
        df['education'] = df['education'].str.replace('9th', 'dropout')
        df['education'] = df['education'].str.replace('HS-Grad', 'HighGrad')
        df['education'] = df['education'].str.replace('HS-grad', 'HighGrad')
        df['education'] = df['education'].str.replace('Some-college', 'CommunityCollege')
        df['education'] = df['education'].str.replace('Assoc-acdm', 'CommunityCollege')
        df['education'] = df['education'].str.replace('Assoc-voc', 'CommunityCollege')
        
        # [' Never-married', ' Married-civ-spouse', ' Divorced',' Married-spouse-absent', ' Separated', ' Married-AF-spouse',
        # ' Widowed']
        df['marital_status'] = df['marital_status'].str.replace('Never-married','notMarried')
        df['marital_status'] = df['marital_status'].str.replace('Seperated','notMarried')
        df['marital_status'] = df['marital_status'].str.replace('Divorced','notMarried')
        df['marital_status'] = df['marital_status'].str.replace('Seperated','notMarried')
        df['marital_status'] = df['marital_status'].str.replace('Married-AF-spouse','Married')
        df['marital_status'] = df['marital_status'].str.replace('Married-civ-spouse','Married')
        df['marital_status'] = df['marital_status'].str.replace('Married-spouse-absent','notMarried')
        
        # df['workclass'] doesn't effect performance
        
        # Binning
        education_num = pd.cut(df.education_num,bins=[0,8,9,12,13,14,15,16],labels=[0,1,2,3,4,5,6])
        df['education_num'] = education_num
        
        # Create categories for continuous values. Run it only once
#         age = pd.cut(df.age,bins=[0,18,27,50,100],labels=[0,1,2,3]) # ['HighGrad','Bachelor','Grad','others'] ==> https://en.wikipedia.org/wiki/Education_in_the_United_States
        
        age = pd.cut(df.age,bins)  
        hours = pd.cut(df.hours_per_week,bins=[0,20,40,49,100],labels=['PT','FT','OverTime','TwoJobs'])
        
        # replace old column with new column with category values
        df['age'] = age
        df['hours_per_week'] = hours
        
        df
        return df
    def take_model(self,model):
        '''
        Takes in model.
        '''
        self.model = model
        
    def split_data(self,df_data,df_label):
        
        self.x_train,self.x_test, self.y_train, self.y_test = model_selection.train_test_split(df_data,df_label,test_size=0.2,random_state=2)

        
    def plain_training(self,df_data,df_label,test_df):
        '''
        It trains model once and returns accuracy score
        '''
        self.x_train,self.x_test, self.y_train, self.y_test = model_selection.train_test_split(df_data,df_label,test_size=0.2,random_state=2)
        self.x_train = self.normalize_data(self.x_train)
        self.x_test = self.normalize_data(self.x_test)
        test = self.normalize_data(test_df)
        clf = self.model()
        clf.fit(self.x_train,self.y_train)
        y_pred = clf.predict(test)
        
        return clf.score(self.x_test,self.y_test)
    
    def generate_submission(self, y_pred):
        '''
        Saves submission in the right format.
        '''
        pred_df = pd.DataFrame()
        pred_df['Id'] = np.arange(0,len(y_pred))
        pred_df['Category'] = y_pred
        pred_df.to_csv('submission.csv',index=False)
        
    def encode_df(self,df):
        '''
        Encode string values in dataframe into numbers.
        '''
        
        return df.apply(LabelEncoder().fit_transform)
    
    def normalize_data(self, X):
        # TO DO: Normalize the feature values of dataset X using the mean and standard deviation of the respective features 
        # min_max has is better than robustscaler
#         min_max_scaler = preprocessing.MinMaxScaler() 
#         X = min_max_scaler.fit_transform(X)
        
#         scaler = preprocessing.RobustScaler()
#         X = scaler.fit_transform(X)
        # scale is better than min_max
#         X = preprocessing.scale(X)
        quantile_transformer = preprocessing.QuantileTransformer(random_state=0)
        X = quantile_transformer.fit_transform(X)
        return X
    
    def kbins(self,df,bins):
        from sklearn.preprocessing import KBinsDiscretizer
        est = KBinsDiscretizer(n_bins=bins,encode='onehot-dense',strategy='kmeans')
        est.fit(df)
        Xt = est.transform(df)
        
        return Xt
    def panda_onehotencode(self,train,test,selected):
        '''
        One hot encode train and test data. Then make sure they have same column amount.
        # https://stackoverflow.com/questions/58101126/using-scikit-learn-onehotencoder-with-a-pandas-dataframe
        https://towardsdatascience.com/encoding-categorical-features-21a2651a065c
        
        returns train, test
        '''
        tmp = pd.get_dummies(train[selected],prefix_sep='_',columns=selected)
        print("train data shape: ",tmp.shape)

        tmp1 = pd.get_dummies(test[selected],prefix_sep='_',columns=selected)
        print("test data shape: ",tmp1.shape)

        final_train, final_test = tmp.align(tmp1, join='inner', axis=1)  # inner join
        print("final train data : {}\nfinal test data: {}".format(final_train.shape,final_test.shape))
        return final_train,final_test
    
    def category_labelencoder(self,data):
        '''
        convert categories into something.
        https://towardsdatascience.com/encoding-categorical-features-21a2651a065c
        
        But I don't know how to make test and train with same column shape.
        '''
        tmp = data
        # print(df_data.head() )
        categorical_feature_mask = tmp.dtypes==object # gets features with string or object value
        categorical_feature_mask
        # filter categorical columns using mask and turn it into a list
        categorical_cols = tmp.columns[categorical_feature_mask].tolist()
        categorical_cols

        # instantiate labelencoder object
        le = LabelEncoder()
        # apply le on categorical feature columns
        tmp[categorical_cols] = tmp[categorical_cols].apply(lambda col: le.fit_transform(col))
        tmp[categorical_cols].head(10)

        # instantiate OneHotEncoder
        ohe = OneHotEncoder(categorical_features = categorical_feature_mask, sparse=False ) 
        # categorical_features = boolean mask for categorical columns
        # sparse = False output an array not sparse matrix

        # apply OneHotEncoder on categorical feature columns
        test_ohe = ohe.fit_transform(tmp) # It returns an numpy array
        return test_ohe
    def kfold_cw(self,model,df_data,df_label,df_test,niter):
        scores = cross_val_score(model,df_data,df_label.values.ravel(),cv=niter)
        print("Accuracy score: ",scores)
#         pred  = cross_val_predict(model,df_data,df_label,cv=niter)
        print('fitting now')
        final_model = model.fit(df_data,df_label.values.ravel())
        print('final prediction')
        new_pred = model.predict(df_test)
        print('acc: ',np.mean(scores))
        return np.mean(scores),new_pred
    

In [311]:
clf = Classification()

# convert train label to dataframe
label = clf.train_label.values.ravel()
df_label = pd.DataFrame(data=label,columns=['income'])
df_label.shape

# train data
df_data = clf.train_data
df_data = clf.preprocess(df_data,20)


# test data
test_df = clf.test_data
test_df = clf.preprocess(test_df,20)



# encode both datas
encoded_df = clf.encode_df(df_data)
encoded_test_df = clf.encode_df(test_df)

# encoded_df = clf.kbins(encoded_df,5)
# encoded_test_df = clf.kbins(encoded_test_df,5)


# normalize train and test data
df_norm = clf.normalize_data(encoded_df)
df_test_norm = clf.normalize_data(encoded_test_df)

# convert from numpy to dataframe
df_norm = pd.DataFrame(data=df_norm,columns=df_data.columns)
df_test_norm = pd.DataFrame(data=df_test_norm,columns=df_data.columns)

# # finds correlation between features
# temp_df = pd.concat([df_norm,clf.train_label],axis=1,sort=False)

# plt.figure(figsize=(12,10))
# corr = temp_df.corr()
# sns.heatmap(corr, annot=True, cmap=plt.cm.Reds)

# #Correlation with output variable
# cor_target = abs(corr["income"])

# #Selecting highly correlated features
# relevant_features = cor_target[cor_target>=0.04]
# selected = list(dict(relevant_features))
# selected = selected[:-1]
# print(selected)
selected = ['age', 'workclass', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country']

final_train = df_norm[selected]
final_test = df_test_norm[selected]

# for i in selected:
#     print(i,df_data[i].unique())
    



In [312]:
# final_train,final_test = clf.panda_onehotencode(final_train,final_test,selected)

In [313]:
# # normalize train and test data
# df_norm = clf.normalize_data(encoded_df[selected])
# df_test_norm = clf.normalize_data(encoded_test_df[selected])

# # convert from numpy to dataframe
# df_norm = pd.DataFrame(data=df_norm,columns=selected)
# df_test_norm = pd.DataFrame(data=df_test_norm,columns=selected)

# print(df_norm)
clf.take_model(ensemble.RandomForestClassifier)
clf.plain_training(final_train,label,final_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8327959465684016

In [271]:
# unique count of prediction
# clf.generate_submission(gradient.predict(df_test_norm))

## Running CV with different method and in the order accuracy score

Below k-fold CV uses 

`cross_val_score`

`cross_val_pred`

In [314]:
# Gradient Boosting
gradient = ensemble.GradientBoostingClassifier(n_estimators=200,learning_rate=0.2)
gdb_score,gdb_pred = clf.kfold_cw(gradient,final_train,df_label,final_test,3)


Accuracy score:  [0.87202875 0.87331859 0.87754538]
fitting now
final prediction
acc:  0.8742975721816584


In [315]:
def compare(pred):
    df = pd.read_csv('submission1.csv')
    mis_classified = 0
    for i in range(len(pred)):
        if(df['Category'][i] != pred[i]):
            mis_classified += 1
    return mis_classified
compare(gdb_pred)

621

In [294]:
# AdaBoostClassifier
adaboost = ensemble.AdaBoostClassifier(n_estimators=200,learning_rate= 1)

ada_score,ada_pred = clf.kfold_cw(adaboost,final_train,df_label,final_test,3)


Accuracy score:  [0.863829   0.86668509 0.87100341]
fitting now
final prediction
acc:  0.867172501793784


In [295]:
compare(ada_pred)

873

In [297]:
# last submission
f = pd.read_csv('submission (3).csv')
compare(f.Category)

796

In [39]:
# clf.generate_submission(gdb_pred)

In [ ]:
# feature_scores1 = pd.Series(adaboost.feature_importances_, index=final_train.columns).sort_values(ascending=False)
# list(dict(feature_scores1[feature_scores1 > 0.009]))

# final_train = final_train[list(dict(feature_scores1[feature_scores1 > 0.009]))]
# final_train

# feature_scores = pd.Series(adaboost.feature_importances_, index=final_test.columns).sort_values(ascending=False)
# list(dict(feature_scores[feature_scores > 0.009]))

# final_test = final_test[list(dict(feature_scores[feature_scores > 0.009]))]
# final_test

In [29]:
# Random forest
rdmforest = ensemble.RandomForestClassifier(n_estimators=200,min_samples_split=2,min_samples_leaf=4,max_depth=10,bootstrap=True)

rdfst_score,rdfst_pred = clf.kfold_cw(rdmforest,final_train,df_label,final_test,3)


Accuracy score:  [0.85562926 0.85995946 0.86418502]
fitting now
final prediction
acc:  0.8599245803395973


In [30]:
rdmforest = ensemble.RandomForestClassifier(n_estimators=200,bootstrap=True)

rdfst_score1,rdfst_pred1 = clf.kfold_cw(rdmforest,final_train,df_label,final_test,10)


Accuracy score:  [0.84065091 0.85534398 0.84305897 0.84459459 0.8470516  0.84643735
 0.84981572 0.85135135 0.86087224 0.84920147]
fitting now
final prediction
acc:  0.8488378178468752


In [31]:
# extra tree classifier
extratree = ensemble.ExtraTreesClassifier(n_estimators=100, max_depth=None,min_samples_split=2, random_state=0)
extra_score, extra_pred = clf.kfold_cw(extratree,final_train,df_label,final_test,3)


Accuracy score:  [0.83591303 0.83480744 0.84400627]
fitting now
final prediction
acc:  0.8382422457547309


In [ ]:
# # Naive Bayes
# NB = naive_bayes.GaussianNB()
# nb_score,nb_pred = clf.kfold_cw(NB,final_train,df_label,final_test,10)


In [32]:
# SVC
from sklearn.svm import SVC
svc = SVC(gamma='auto')
svc_score,svc_pred = clf.kfold_cw(svc,final_train,df_label,final_test,3)


Accuracy score:  [0.83342547 0.8378478  0.8417949 ]
fitting now
final prediction
acc:  0.8376893862445618


In [ ]:
# clf.generate_submission(gdb_pred)

In [ ]:
# Decision Tree
decisiontree = tree.DecisionTreeClassifier(max_depth=None,min_samples_split=2,random_state=0)
tree_score,tree_pred = clf.kfold_cw(decisiontree,final_train,df_label,final_test,10)


In [ ]:
# Lasso
# lasso = linear_model.Lasso(alpha=0.7)
# lasso_score, lasso_pred = clf.kfold_cw(lasso,final_train,df_label,final_test,3)


In [ ]:
bagging = ensemble.BaggingClassifier(n_estimators=10,base_estimator=SVC(gamma='auto'),random_state=0)
score,pred = clf.kfold_cw(bagging,final_train,df_label,final_test,3)

# accuracy of [0.82725263 0.83185922 0.83165945]

With normalization and 3 kfold cv, the random forest with n_estimators=1600,min_samples_split=2,min_samples_leaf=4,max_depth=10,bootstrap=True parameters 

gave this accuracy scores

[0.83618942 0.83812419 0.84059707]

And with minmaxscaler and svr model ```best_svr = SVR(kernel='rbf',gamma='auto') ```, we got

[0.85185185 0.85756403 0.86289505]

In [ ]:
np.unique(ada_pred,return_counts=True)

In [ ]:
np.unique(extra_pred,return_counts=True)

In [ ]:
np.unique(rdfst_pred,return_counts=True)

In [ ]:
np.unique(gdb_pred,return_counts=True)

https://stats.stackexchange.com/questions/411290/how-to-use-a-cross-validated-model-for-prediction

In [ ]:
# clf.generate_submission(gradient.predict(final_test))

## Random Forest Classifier

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

In [ ]:
score = []
pred = []
step_size = [0.1,0.3,0.5,0.6,0.7,0.8,0.9,1,1.5,1.8]
actual_pred = []
for n,s in zip(random_grid['n_estimators'],step_size):
    adaboost = ensemble.AdaBoostClassifier(n_estimators=n,learning_rate=s)

    ada_score,ada_pred = clf.kfold_cw(adaboost,final_train,df_label,final_test,3)
    score.append(ada_score)
    pred.append(np.unique(ada_pred,return_counts=True))
    actual_pred.append(ada_pred)
    print(ada_score)
    

In [ ]:
data_info = pd.DataFrame()
data_info['n_estimators'] = random_grid['n_estimators']
data_info['step_size'] = step_size
data_info['score'] = score
data_info['pred_count'] = pred
data_info['actual_pred'] = actual_pred
data_info

In [ ]:
data_info[data_info.score == data_info['score'].max()]mis_classified

In [ ]:
score1 = []
pred1 = []
step_size = [0.1,0.3,0.5,0.6,0.7,0.8,0.9,1,1.5,1.8]
for n,s in zip(random_grid['n_estimators'],step_size):
    gradient = ensemble.GradientBoostingClassifier(n_estimators=n,learning_rate=s)
    gdb_score,gdb_pred = clf.kfold_cw(gradient,final_train,df_label,final_test,3)
    print(gdb_score)
    score1.append(gdb_score)
    pred1.append(np.unique(gdb_pred,return_counts=True))


In [ ]:
# # https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]

# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(final_train, df_label)